In [1]:
%matplotlib inline
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from preprocessing import *
import cv2


from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Lambda
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import time


# to autoreload modules on code change
%load_ext autoreload
%autoreload 2

/home/u8710/.conda/envs/my_root/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(
#         featurewise_center=True,
#         featurewise_std_normalization=True,
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
        rescale=1./255,
#         shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
#         preprocessing_function= lambda x: cv2.resize(x,(150,150))
)

test_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
#         preprocessing_function= lambda x: cv2.resize(x,(150,150))
)


In [3]:
trn_path = './data/train/'
val_path = './data/val/'
bs = 32


dl = datagen.flow_from_directory(trn_path,
                                 target_size=(200,200),
                                 class_mode='categorical',
                                 batch_size=32,
                                 shuffle=True)

test_dl = datagen.flow_from_directory(val_path,
                       target_size=(200,200),
                       class_mode='categorical',
                       batch_size=32,
                       shuffle=True)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [4]:
shp = (200,200,3)
model = ResNet50(weights='imagenet',include_top=False,input_shape=shp)

In [5]:
for layer in model.layers:
    layer.trainable = False

In [6]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(7, activation="softmax")(x)

In [7]:
model2 = Model(inputs = model.input, outputs = predictions)

model2.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [15]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 100, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [8]:
def sclr(x,y):
    return y*0.95

In [10]:
fp = 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(fp, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
# lr_s = LearningRateScheduler(sclr, verbose=1)

In [ ]:
model2.fit_generator(dl,
                    steps_per_epoch = 5,
                    epochs = 10,
                    validation_data = test_dl,
                    validation_steps = 2,
                    use_multiprocessing=True,
                    callbacks = [checkpoint, early])

Epoch 1/10
4/5 [=======================>......] - ETA: 3s - loss: 3.0132 - acc: 0.1094
Epoch 00001: val_acc improved from -inf to 0.34375, saving model to weights.01-1.97.hdf5
5/5 [==============================] - 28s 6s/step - loss: 3.0504 - acc: 0.1250 - val_loss: 1.9683 - val_acc: 0.3438
Epoch 2/10
